# 2.3: ETL
Deze notebook kan 2 dingen doen:
1. Alle data in de data warehouse schoonmaken
2. Alle data vanuit de 3 sqlite DB's en de 2 CSV-bestanden vullen in de DW

In [35]:
import pyodbc
import sqlite3
import pandas as pd
import sqlalchemy as sa

connection_url = sa.engine.URL.create(
    'mssql+pyodbc',
    username='SA',
    password='MyPass@word',
    host='127.0.0.1',
    database='deds',
    query={"driver": "ODBC Driver 17 for SQL Server", "Encrypt": "no"}
)

engine = sa.create_engine(connection_url)

conn = engine.connect()
print("Connected successfully!")
conn.close()

Connected successfully!


## Clear Data

In [40]:
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import sqlalchemy as sa

Session = sessionmaker(bind=engine)
session = Session()

# Step 2: Disable foreign key constraints (SQL Server-specific)
session.execute(text("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
session.commit()

# Step 3: Delete all data from all tables
meta = sa.MetaData()
meta.reflect(bind=engine)  # Load table information

with session.begin():  # Ensures transaction safety
    for table in reversed(meta.sorted_tables):  # Drop data respecting dependencies
        session.execute(table.delete())  # `DELETE FROM table`

# Step 4: Re-enable foreign key constraints
session.execute(text("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'"))
session.commit()

# Close session
session.close()

## Fill Data

### CRM

In [41]:
conn = sqlite3.connect("../data/go_crm_train.sqlite")

age_group = pd.read_sql_query('SELECT * FROM age_group', conn)
age_group.to_sql('age_group', engine, if_exists='append', index=False)

sales_territory = pd.read_sql_query('SELECT * FROM sales_territory', conn)
sales_territory.to_sql('sales_territory', engine, if_exists='append', index=False)

country = pd.read_sql_query('SELECT * FROM country', conn)
country['LANGUAGE'] = 'EN'
country['CURRENCY_NAME'] = 'temp'
country.to_sql('country', engine, if_exists='append', index=False)

retailer_segment = pd.read_sql_query('SELECT * FROM retailer_segment', conn)
retailer_segment.to_sql('retailer_segment', engine, if_exists='append', index=False)

retailer_headquarters = pd.read_sql_query('SELECT * FROM retailer_headquarters', conn)
retailer_headquarters[:1]

# .to_sql('retailer_headquarters', engine, if_exists='append', index=False)

# retailer_type = pd.read_sql_query('SELECT * FROM retailer_type', conn)
# retailer_type.to_sql('retailer_type', engine, if_exists='append', index=False)

# retailer = pd.read_sql_query('SELECT * FROM retailer', conn)
# retailer.to_sql('retailer', engine, if_exists='append', index=False)

# retailer_contact = pd.read_sql_query('SELECT * FROM retailer_contact', conn)
# retailer_contact.to_sql('retailer_contact', engine, if_exists='append', index=False)

# retailer_site = pd.read_sql_query('SELECT * FROM retailer_site', conn)
# retailer_site.to_sql('retailer_site', engine, if_exists='append', index=False)

# sales_demographic = pd.read_sql_query('SELECT * FROM sales_demographic', conn)
# sales_demographic.to_sql('sales_demographic', engine, if_exists='append', index=False)

,RETAILER_CODEMR,RETAILER_NAME,ADDRESS1,ADDRESS2,CITY,REGION,POSTAL_ZONE,COUNTRY_CODE,PHONE,FAX,SEGMENT_CODE
0,1,Rock Steady,"45, rue Atwater",None,Montréal,Québec,H2T 9K8,4,1 (514) 747-4481,1 (514) 741-2277,2


### Sales

In [25]:
conn = sqlite3.connect("../data/go_sales_train.sqlite")

product = pd.read_sql_query('SELECT * FROM product', conn)



,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,1,1995-02-15,1,4.00,0.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ..."
1,2,1995-02-15,1,9.22,0.23,P02CE1CG1.jpg,EN,TrailChef Canteen,Aluminum canteen. Rugged fleece-lined cover wi...
2,3,1995-02-15,1,15.93,0.28,P03CE1CG1.jpg,EN,TrailChef Kitchen Kit,"Zippered nylon pouch contains cutlery for two,..."
3,4,1995-02-15,1,5.00,0.28,P04CE1CG1.jpg,EN,TrailChef Cup,Tin cup. Holds 0.4 liters. Weight: 60 g
4,5,1995-02-15,1,34.97,0.30,P05CE1CG1.jpg,EN,TrailChef Cook Set,All you will ever need on the trail. Pot grip...


### Staff

In [ ]:
conn = sqlite3.connect("../data/go_staff_train.sqlite")



### Product Forecast

### Inventory Levels